In [125]:
import numpy as np

**MAPA**

Definição das dimensões do mapa e inicialização das qualidades para cada estado

Definição das possíveis ações a serem tomadas


*   Ir para cima = índice 0
*   Ir para a direita = Índice 1
*   Ir para baixo = índice 2
*   Ir para a esquerda = Índice 3

In [126]:
mapa_linhas = 3
mapa_colunas = 4

matrizQ = np.zeros ((mapa_linhas, mapa_colunas, 4))
tedio = np.ones ((mapa_linhas, mapa_colunas, 4))
visitas = np.zeros ((mapa_linhas, mapa_colunas, 4))

acoes = ['cima', 'direita', 'baixo', 'esquerda']

**RECOMPENSAS**

Definição das recompensas de cada estado

In [127]:
recompensas = np.full ((mapa_linhas, mapa_colunas), -1)
recompensas[0, 3] = 100
recompensas[1, 1] = -100

for i in recompensas:
    print (i)    

[ -1  -1  -1 100]
[  -1 -100   -1   -1]
[-1 -1 -1 -1]


**Funções Auxiliares**

In [128]:
def chegou_destino (linha, coluna):
    ''' print ("linha  == ", linha, " coluna == ", coluna) '''
    if recompensas[linha, coluna] == 100:
        return True
    return False

def escolhe_acao (linha, coluna, taxa):
    x = np.random.random ()
    if x <= taxa:
        ''' y = np.argmax (matrizQ [linha, coluna])  '''
        if np.count_nonzero (np.max (matrizQ [linha, coluna])) > 1:
            y = np.argmax (np.abs (tedio [linha, coluna]))
        else:
            y = np.argmax (matrizQ [linha, coluna])
    else:
        y = np.random.randint(4)
    visitas [linha, coluna, y] += 1
    return y

def valida_acao (linha, coluna, acao):
    # Tenta ir para cima
    if acao == 0 and linha > 0:
        return True
    # Tenta ir para a direita
    elif acao == 1 and coluna < mapa_colunas - 1:
        return True
    # Tenta ir para baixo
    elif acao == 2 and linha < mapa_linhas - 1:
        return True
    # Tenta ir para a esquerda
    elif acao == 3 and coluna > 0:
        return True
    # Não consegue fazer a ação
    ''' print (linha, coluna, acao)
    matrizQ [linha, coluna, acao] = -10 '''
    ''' print ("Matriz Q antes ", matrizQ)
    print ("Matriz Q depois \n", matrizQ) '''
    return False

def realiza_acao (linha, coluna, acao):
    nova_linha, nova_coluna = linha, coluna
    if acao == 0:
        nova_linha -= 1
    elif acao == 1:
        nova_coluna += 1
    elif acao == 2:
        nova_linha += 1
    elif acao == 3:
        nova_coluna -= 1
    return nova_linha, nova_coluna

  
def menor_prejuizo (lin_inicio, col_inicio):
    if chegou_destino (lin_inicio, col_inicio):
        return []
    
    cur_row = lin_inicio
    cur_col = col_inicio
    caminho = []
    caminho.append ([cur_row, cur_col])

    while not chegou_destino (cur_row, cur_col):
        acao = escolhe_acao (cur_row, cur_col, 1.)
        if valida_acao (cur_row, cur_col, acao):
            cur_row, cur_col = realiza_acao (cur_row, cur_col, acao)
            caminho.append ([cur_row, cur_col])

    return caminho

In [129]:
def treina ():
    print ("Matriz de recompensas: ")
    print (recompensas)
    learning_rate = 0.9
    desconto = 0.9
    chance = 0.8

    posicao_inicial = [2, 0]
    posicao_final = [0, 3]

    ''' print ("linha atual == ", lin_atual, " coluna atual == ", col_atual) '''

    for episodios in range (20):
        lin_atual = lin_ant = posicao_inicial[0]
        col_atual = col_ant = posicao_inicial[1]
        caminho = [posicao_inicial]

        while not chegou_destino (lin_atual, col_atual):
            nova_acao = escolhe_acao (lin_atual, col_atual, chance)
            ''' print ("nova ação == ", nova_acao) '''
            lin_ant = lin_atual
            col_ant = col_atual

            if valida_acao (lin_atual, col_atual, nova_acao):
                lin_atual, col_atual = realiza_acao (lin_atual, col_atual, nova_acao)
            
                rec = recompensas[lin_atual, col_atual]
                q_anterior = matrizQ [lin_ant, col_ant, nova_acao]
                diferenca_temporal = rec + desconto * np.max(matrizQ [lin_atual, col_atual]) - q_anterior
                matrizQ [lin_ant, col_ant, nova_acao] = q_anterior + learning_rate * diferenca_temporal
                if visitas [lin_ant, col_ant, nova_acao] != 0:
                    tedio [lin_ant, col_ant, nova_acao] = matrizQ [lin_ant, col_ant, nova_acao] / visitas [lin_ant, col_ant, nova_acao]

                caminho.append ([lin_atual, col_atual])
            
        
        ''' print ("Caminho no episodio ", episodios, " == ", caminho)
        print (caminho)
        if len(caminho) < 10:
            print ("Tamanho do caminho == ", len (caminho))
            if len(caminho) == 6:
                print ("Episodio ", episodios)
                print ("Chegou no ponto final em 6 passos")
                print ("Vezes que passou por [2, 3] == ", caminho.count ([2, 3]))
                if caminho.count ([1, 1]) > 0: 
                    print ("Mas passou por [1, 1] ou não passou uma vez em [2, 3] :(")
                    caminho.clear ()
                else:
                    break
        if len(caminho) != 6:
            caminho.clear () '''
            

    print (matrizQ)
    print ("Matriz de tedio: ")
    print (tedio)
    print ("Matriz de visitas: ")
    print (visitas)

In [130]:
treina ()

print ("\nCaminho escolhido pelo algoritmo: ")
print (menor_prejuizo (2, 0))

Matriz de recompensas: 
[[  -1   -1   -1  100]
 [  -1 -100   -1   -1]
 [  -1   -1   -1   -1]]
[[[  0.          79.086492    -0.999        0.        ]
  [  0.          88.99983      0.          -0.9       ]
  [  0.         100.          78.22305899   0.        ]
  [  0.           0.           0.           0.        ]]

 [[ 69.8574879  -28.91660135  46.34449503   0.        ]
  [ 57.33        79.1         -0.999        0.        ]
  [ 89.          79.2         -0.9        -28.52190342]
  [ 99.           0.           0.          57.42      ]]

 [[ 59.90146627  -1.           0.           0.        ]
  [-28.81000014  -0.999999     0.          -1.        ]
  [ -0.99        -0.99         0.          -0.9999    ]
  [ -0.9          0.           0.          -0.9       ]]]
Matriz de tedio: 
[[[ 1.00000000e+00  1.58172984e+01 -3.33000000e-01  1.00000000e+00]
  [ 1.00000000e+00  1.48333050e+01  1.00000000e+00 -9.00000000e-01]
  [ 1.00000000e+00  5.55555556e+00  3.91115295e+01  1.00000000e+00]
  [ 1.

**Recompensa Positiva no Mapa**

Aqui fazemos o mesmo processo feito anteriormente mas alteramos o mapa para conter a recompensa positiva

In [131]:
matrizQ = np.zeros ((mapa_linhas, mapa_colunas, 4))
tedio = np.ones ((mapa_linhas, mapa_colunas, 4))
visitas = np.zeros ((mapa_linhas, mapa_colunas, 4))
recompensas[2, 3] = 10

treina ()

print (menor_prejuizo (2, 0))

Matriz de recompensas: 
[[  -1   -1   -1  100]
 [  -1 -100   -1   -1]
 [  -1   -1   -1   10]]
[[[  0.          -0.99        -0.9          0.        ]
  [  0.          -0.99       -34.9820448   -0.9       ]
  [  0.          90.          -0.99        -0.9       ]
  [  0.           0.           0.           0.        ]]

 [[ -0.99       -29.41812833  56.07029638   0.        ]
  [ -0.99        79.07231836   0.           0.        ]
  [ -0.99        88.99999729  -0.9        -25.9820448 ]
  [100.           0.          85.45804852  79.02025223]]

 [[ -0.99999999  71.08099982   0.           0.        ]
  [-90.          80.08999999   0.           0.        ]
  [ 71.09997808  90.1          0.           0.        ]
  [ 89.           0.           0.          -0.9       ]]]
Matriz de tedio: 
[[[  1.          -0.495       -0.9          1.        ]
  [  1.          -0.495      -17.4910224   -0.9       ]
  [  1.          90.          -0.495       -0.9       ]
  [  1.           1.           1.         